# Preprocessing of WOD18 data
    - merge osd and ctd data

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sklearn as skl
import gsw
import cartopy.crs as ccrs
from scipy.interpolate import interp1d

### First prepare O2, T, S, stratification data from WOD

In [7]:
# observational data 
diro = '/glade/scratch/ito/WOD18_OSDCTD/'
fosd='_1x1bin_osd_'
fctd='_1x1bin_ctd_'
vs=['t','s','o2']

In [8]:
# basin mask
dsm=xr.open_dataset('/glade/work/ito/dataset/model/basin_mask_01.nc')

In [9]:
def merge_osd_ctd(year):
    for n in range(3):
        if year < 1987:
            d0=xr.open_dataset(diro+vs[n]+fosd+str(year)+'.nc')
            d0.to_netcdf(diro+vs[n]+'_1x1bin_merged_'+str(year)+'.nc')
        else:
            d0=xr.open_dataset(diro+vs[n]+fosd+str(year)+'.nc')
            d1=xr.open_dataset(diro+vs[n]+fctd+str(year)+'.nc')
            N0=d0.sample_count.to_numpy()
            N1=d1.sample_count.to_numpy()
            if n == 0:
                c0=d0.t.to_numpy()
                c1=d1.t.to_numpy()
            elif n == 1:
                c0=d0.s.to_numpy()
                c1=d1.s.to_numpy()
            else:
                c0=d0.o2.to_numpy()
                c1=d1.o2.to_numpy()
            #
            mon=d0.mon.to_numpy()
            depth=d0.depth.to_numpy()
            lat=d0.lat.to_numpy()
            lon=d0.lon.to_numpy()
            #
            tmp=(c0*N0 + c1*N1)/(N0+N1)
            da=xr.DataArray(data=tmp,name=vs[n],dims=['mon','depth','lat','lon'],
                        coords={'mon':mon,'depth':depth,'lat':lat,'lon':lon})
            ds=da.to_dataset()
            N=N0+N1
            ds['sample_count']=xr.DataArray(data=N,name='sample_count',\
                        dims=['mon','depth','lat','lon'],\
                        coords={'mon':mon,'depth':depth,'lat':lat,'lon':lon})
            #
            ds.to_netcdf(diro+vs[n]+'_1x1bin_merged_'+str(year)+'.nc')
    return 1

In [10]:
yrs=np.arange(1965,2021,1)
from multiprocessing import Pool
if __name__ == '__main__':
    with Pool(8) as p:
        print(p.map(merge_osd_ctd, yrs))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
